 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#VGG16-model" data-toc-modified-id="VGG16-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>VGG16 model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Embedding-of-COCO-images" data-toc-modified-id="Embedding-of-COCO-images-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Embedding of COCO images</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#LSTM-+-VGG" data-toc-modified-id="LSTM-+-VGG-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LSTM + VGG</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Index-answers" data-toc-modified-id="Index-answers-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Index answers</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-text-data" data-toc-modified-id="Prepare-text-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Prepare text data</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-the-embedding-layer" data-toc-modified-id="Prepare-the-embedding-layer-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Prepare the embedding layer</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-image-data-I" data-toc-modified-id="Prepare-image-data-I-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Prepare image data I</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-image-data-II-(pass-through-VGG)" data-toc-modified-id="Prepare-image-data-II-(pass-through-VGG)-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Prepare image data II (pass through VGG)</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Sanity-check" data-toc-modified-id="Sanity-check-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Sanity check</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-definition" data-toc-modified-id="Model-definition-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Model definition</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-I-(224x224x3-input)" data-toc-modified-id="Model-I-(224x224x3-input)-2.7.1"><span class="toc-item-num">2.7.1&nbsp;&nbsp;</span>Model I (224x224x3 input)</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-II-(4096-input-from-VGG)" data-toc-modified-id="Model-II-(4096-input-from-VGG)-2.7.2"><span class="toc-item-num">2.7.2&nbsp;&nbsp;</span>Model II (4096 input from VGG)</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Training" data-toc-modified-id="Training-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Predicting-on-test-data-(val2014)" data-toc-modified-id="Predicting-on-test-data-(val2014)-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Predicting on test data (val2014)</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Preparing-test-data" data-toc-modified-id="Preparing-test-data-2.9.1"><span class="toc-item-num">2.9.1&nbsp;&nbsp;</span>Preparing test data</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# Add the path of the main directory
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import keras

from keras.layers import Dense, Dropout, LSTM, multiply
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.layers import Input
from keras.models import Model



Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
from vqa_api.PythonHelperTools.vqaTools.vqa import VQA

In [4]:
%load_ext autoreload

%autoreload 2

# VGG16 model

In [5]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tools import img_dir, img_file

In [6]:
# VGG16 model
vgg_model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', 
                               input_tensor=None, input_shape=None, 
                               pooling=None, classes=1000)

In [7]:
# Model layers 
layers = vgg_model.layers
for i in range(len(layers)):
    print "--> Layer {}".format(i+1), "\n"
    print layers[i].get_config(), "\n"
    

--> Layer 1 

{'dtype': 'float32', 'batch_input_shape': (None, 224, 224, 3), 'name': 'input_1', 'sparse': False} 

--> Layer 2 

{'padding': 'same', 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'block1_conv1', 'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'activation': 'relu', 'trainable': True, 'data_format': 'channels_last', 'kernel_constraint': None, 'strides': (1, 1), 'dilation_rate': (1, 1), 'kernel_regularizer': None, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'use_bias': True, 'activity_regularizer': None, 'kernel_size': (3, 3)} 

--> Layer 3 

{'padding': 'same', 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'block1_conv2', 'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'activation': 'relu', 'trainable': True, 'data_

The fc7 layer is the layer 22, which name is fc2. 

In [ ]:
# fc7 layer
fc7_layer = layers[21]
print fc7_layer.get_config()
fc7_layer_name = fc7_layer.get_config()["name"]
print fc7_layer_name

In [ ]:
# Input shape and output shape of the fc7 layer
print "Input shape:", fc7_layer.input_shape
print "Ouput shape:", fc7_layer.output_shape

In [ ]:
# Weights of the fc7 layer
# Weights shape
print "Weights shape:", fc7_layer.get_weights()[0].shape
# Bias shape
print "Bias shape:", fc7_layer.get_weights()[1].shape

We can also get a Layer by its name:

In [ ]:
# fc7 layer
fc7_layer = vgg_model.get_layer(fc7_layer_name)
fc7_layer.get_config()

## Embedding of COCO images 

In [ ]:
dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"
dataType = "mscoco"

In [ ]:
image_ids = [(9, "train2014"), (25, "train2014")]
# List to store the arrays for images
imgs = []
for image_id in image_ids:
    # Resize the images as VGG inputs
    img = image.load_img(os.path.join(img_dir(dataDir, dataType, 
        image_id[1]), img_file(image_id[1], image_id[0])), target_size=(224, 224)) 
    img = image.img_to_array(img)
    imgs.append(img)
imgs = np.stack(imgs)
# Preprocess the images corresponding to VGG
imgs = preprocess_input(imgs)   

In [ ]:
img = img.reshape(1, 224, 224, 3)

In [ ]:
img.shape

In [ ]:
# "Submodel" of VGG until the fc7 layer
vgg_model_fc7 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

In [ ]:
# Images embedding using the fc7 layer of VGG16
vgg_model_fc7.predict(img).flatten().shape

# LSTM + VGG

In [8]:
from lstm_vgg import LSTMVGG

In [9]:
dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"
# dataDir = "/home/ubuntu/data/vqa"

In [10]:
text_vision = LSTMVGG(dataDir)

C:\Users\Nasser Benab\Documents\git\data\vqa\Annotations\mscoco_train2014_annotations.json
--> train2014
loading VQA annotations and questions into memory...
0:00:12.404000
creating index...
index created!
loading VQA annotations and questions into memory...
0:00:05.289000
creating index...
index created!


## Index answers

In [11]:
text_vision.get_top_answers_train()

Getting the top 1000 answers from the training set ...
Mapping each top answer to an index between 0 and 999 in self.idx_to_answer_dic and self.answer_to_idx_dic ...


In [12]:
text_vision.get_most_common_answer_train()

Getting the ground truth answer for each question/image ...


In [13]:
text_vision.reduce_answers_train()

Keeping only every question/image with an answer among the top 1000 answers of the training set ...


In [14]:
text_vision.encode_answers_train()

Encoding train answers ...


In [15]:
text_vision.train_answers_categorical.shape

(214477L, 1000L)

## Prepare text data 

In [16]:
# Tokenize questions using the vocabulary of the training questions
text_vision.tokenize_questions_train()

Fitting the tokenizer on the training questions ...
Embedding the train questions ...
Padding the sequences to a maximum length of 22 ...


In [17]:
# Number of questions in the train set
len(text_vision.train_sequences)

214477

## Prepare the embedding layer

In [18]:
text_vision.create_embedding_matrix()

In [19]:
text_vision.embedding_matrix.shape

(12494L, 300L)

## Prepare image data I

In [ ]:
text_vision.process_images_train(n=300)

In [ ]:
text_vision.train_images.shape

## Prepare image data II (pass through VGG)

In [20]:
%time text_vision.process_encode_images_train(n=10)

Images have already been encoded, opening them ...
Wall time: 7 ms


In [21]:
text_vision.train_images.shape

(10L, 4096L)

## Sanity check  

Let's check that the different data correspond (question, answer and image):

In [ ]:
i = 0

In [ ]:
text_vision.train_questions[i]

In [ ]:
text_vision.train_answers[i]

In [ ]:
# plt.imshow(text_vision.train_images[i])
# plt.show()

## Model definition

In [22]:
# Maximum length for the input questions
input_length = 22

# Vocabulary size from the training set
V = text_vision.vocabulary_size_train
input_dim = V + 1

# Dimension of the vectors (from the paper)
embedding_dim = 300

# The output from the embedding layer is equal to input_length x embedding_dim

### Model I (224x224x3 input)

In [ ]:
# VGG16 model
vgg_model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', 
                               input_tensor=None, input_shape=None, 
                               pooling=None, classes=1000)

In [ ]:
# "Submodel" of VGG until the fc7 layer
vgg_model_fc7 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

In [ ]:
# Let"s get a tensor with the output of our vision model
image_input = Input(shape=(224, 224, 3))
# Freeze the vision model weights
vgg_model_fc7.trainable = False
fc7 = vgg_model_fc7(image_input)

# Turn the 4096 embedding from the fc7 layer to a 1024 embedding in order to 
# match the questions embedding
encoded_image = Dense(1024, activation="tanh")(fc7)

# Langage model
question_input = Input(shape=(22,), dtype="int32")
embedded_question = Embedding(input_dim=input_dim, output_dim=embedding_dim,
                              weights=[text_vision.embedding_matrix],
                              input_length=input_length,
                              trainable=False
                             )(question_input)
encoded_question = LSTM(1024)(embedded_question)

# Point-wise multiplication of the outputs from the vision model and the 
# langage model
merge = multiply([encoded_question, encoded_image])

# Hidden layers with 0.5 dropouts
merge = Dropout(0.5)(merge)
hidden1 = Dense(1000, activation="tanh")(merge)
hidden1 = Dropout(0.5)(hidden1)
hidden2 = Dense(1000, activation="tanh")(hidden1)

# Final softmax layer
output = Dense(1000, activation="softmax")(hidden2)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# Summarize layers
print(vqa_model.summary())
# Plot graph
# plot_model(model, to_file="vision_model.png")

###  Model II (4096 input from VGG)

In [23]:
# Let"s get a tensor with the output of our vision model
image_input = Input(shape=(4096,))

# Turn the 4096 embedding from the fc7 layer to a 1024 embedding in order to 
# match the questions embedding
encoded_image = Dense(1024, activation="tanh")(image_input)

# Langage model
question_input = Input(shape=(22,), dtype="int32")
embedded_question = Embedding(input_dim=input_dim, output_dim=embedding_dim,
                              weights=[text_vision.embedding_matrix],
                              input_length=input_length,
                              trainable=False
                             )(question_input)
encoded_question = LSTM(1024)(embedded_question)

# Point-wise multiplication of the outputs from the vision model and the 
# langage model
merge = multiply([encoded_question, encoded_image])

# Hidden layers with 0.5 dropouts
merge = Dropout(0.5)(merge)
hidden1 = Dense(1000, activation="tanh")(merge)
hidden1 = Dropout(0.5)(hidden1)
hidden2 = Dense(1000, activation="tanh")(hidden1)

# Final softmax layer
output = Dense(1000, activation="softmax")(hidden2)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# Summarize layers
print(vqa_model.summary())
# Plot graph
# plot_model(model, to_file="vision_model.png")

C:\Users\Nasser Benab\Anaconda2\lib\site-packages\keras\layers\recurrent.py:1923: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 22, 300)      3748200     input_3[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1024)         5427200     embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (D

## Training 

In [26]:
# Delete variables 
text_vision.clear_variables()

In [27]:
vqa_model.compile(optimizer="rmsprop", loss="categorical_crossentropy",
              metrics=["accuracy"])

In [28]:
# Split train into a train and val set 
split_index_1 = 8
split_index_2 = 10

In [29]:
x_train = [text_vision.train_images[:split_index_1], text_vision.train_sequences[:split_index_1]]
x_val = [text_vision.train_images[split_index_1:split_index_2], text_vision.train_sequences[split_index_1:split_index_2]]
y_train = text_vision.train_answers_categorical[:split_index_1]
y_val = text_vision.train_answers_categorical[split_index_1:split_index_2]

In [30]:
print x_train[0].shape, x_train[1].shape, y_train.shape
print x_val[0].shape, x_val[1].shape, y_val.shape

(8L, 4096L) (8L, 22L) (8L, 1000L)
(2L, 4096L) (2L, 22L) (2L, 1000L)


In [32]:
history = vqa_model.fit(x_train, y_train, validation_data=(x_val, y_val),
              epochs=10, batch_size=128)

Train on 8 samples, validate on 2 samples
Epoch 1/10
8/8 [==============================] - 3s 406ms/step - loss: 6.6294 - acc: 0.6250 - val_loss: 10.1096 - val_acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 4s 442ms/step - loss: 2.5310 - acc: 0.2500 - val_loss: 4.8655 - val_acc: 0.5000
Epoch 3/10
8/8 [==============================] - 3s 403ms/step - loss: 5.2921 - acc: 0.3750 - val_loss: 10.0886 - val_acc: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 3s 385ms/step - loss: 6.0662 - acc: 0.3750 - val_loss: 8.0661 - val_acc: 0.5000
Epoch 5/10
8/8 [==============================] - 3s 385ms/step - loss: 6.3696 - acc: 0.3750 - val_loss: 10.6148 - val_acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 3s 384ms/step - loss: 4.3257 - acc: 0.3750 - val_loss: 8.0655 - val_acc: 0.5000
Epoch 7/10
8/8 [==============================] - 3s 398ms/step - loss: 3.5268 - acc: 0.3750 - val_loss: 10.7183 - val_acc: 0.0000e+00
Epoch 8/10
8/8 [============

In [ ]:
# Summarize history for accuracy
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

# Summarize history for loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
# Save the model 
vqa_model.save(os.path.join(text_vision.dataDir, "Model", "lstm_vgg.h5"))

In [ ]:
# Load the model
from keras.models import load_model
vqa_model = load_model(os.path.join(text_vision.dataDir, "Model", "lstm_vgg.h5"))

## Predicting on test data (val2014)

### Preparing test data 

In [33]:
# Tokenize questions using the vocabulary of the training questions
text_vision.tokenize_questions_test()

Embedding the test questions ...
Padding the sequences to a maximum length of 22 ...


In [34]:
# Number of questions in the train set
len(text_vision.test_sequences)

121512

In [35]:
%time text_vision.process_encode_images_test(n=10)

Images have already been encoded, opening them ...
Wall time: 10 ms


In [38]:
text_vision.test_images.shape

(10L, 4096L)

In [48]:
subset = 10
x_test = [text_vision.test_images[:subset], 
         text_vision.test_sequences[:subset]]


In [49]:
predictions = vqa_model.predict(x_test)

In [52]:
res = text_vision.predictions_to_dic(predictions, text_vision.test_questions_ids[:subset])
res

[{'answer': u'yes', 'question_id': 3506232},
 {'answer': u'yes', 'question_id': 3506230},
 {'answer': u'yes', 'question_id': 3506231},
 {'answer': u'no', 'question_id': 86472},
 {'answer': u'no', 'question_id': 86470},
 {'answer': u'no', 'question_id': 86471},
 {'answer': u'no', 'question_id': 4344100},
 {'answer': u'no', 'question_id': 4344101},
 {'answer': u'no', 'question_id': 4344102},
 {'answer': u'no', 'question_id': 562050}]

In [56]:
text_vision.annotations_test[0]

{u'answer_type': u'other',
 u'answers': [{u'answer': u'wood',
   u'answer_confidence': u'yes',
   u'answer_id': 1},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 2},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 3},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 4},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 5},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 6},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 7},
  {u'answer': u'wood', u'answer_confidence': u'maybe', u'answer_id': 8},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 9},
  {u'answer': u'wood', u'answer_confidence': u'yes', u'answer_id': 10}],
 u'image_id': 350623,
 u'multiple_choice_answer': u'wood',
 u'question_id': 3506232,
 u'question_type': u'what is the'}

In [46]:
# answers_ind_val_true = np.argmax(y_val, axis=1)
# idx_to_answer = np.vectorize(text_vision.idx_to_answer)
# idx_to_answer(answers_ind_val_true)

array([u'yes', u'vegetables'],
      dtype='<U10')